In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

## Data

In [ ]:
data = pd.read_csv('data/heart.csv')

data.info()

## Data Processing

In [ ]:
data.dropna(inplace = True)

In [ ]:
def outlier(table, column: str):
    column_z = column + 'Z'
    table[column_z] = (table[column] - table[column].mean()) / table[column].std()

    index = data[ data[column_z] > 4].index
    data.drop(index, inplace = True)
    index = data[ data[column_z] < -4].index
    data.drop(index, inplace = True)

    data.drop(columns = column_z, inplace = True)
    return

In [ ]:
outlier(data, 'Age')
outlier(data, 'Oldpeak')
outlier(data, 'MaxHR')
outlier(data, 'RestBP')
outlier(data, 'Chol')
data.head()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 15)
sns.pairplot(data.drop(columns='Unnamed: 0'))

In [ ]:
data['Age'] = (data['Age'] - data['Age'].min()) / (data['Age'].max() - data['Age'].min())
data['RestBP'] = (data['RestBP'] - data['RestBP'].min()) / (data['RestBP'].max() - data['RestBP'].min())
data['Chol'] = (data['Chol'] - data['Chol'].min()) / (data['Chol'].max() - data['Chol'].min())
data['MaxHR'] = (data['MaxHR'] - data['MaxHR'].min()) / (data['MaxHR'].max() - data['MaxHR'].min())

data.describe()

In [ ]:
AHD_dict = {AHD:i for i,AHD in enumerate(data['AHD'].unique())}

def encode_ahd(AHD):
  return AHD_dict[AHD]

data['AHD'] = data['AHD'].apply(encode_ahd)

In [ ]:
Thal_dict = {Thal:i for i,Thal in enumerate(data['Thal'].unique())}

def encode_Thal(Thal):
  return Thal_dict[Thal]

data['Thal'] = data['Thal'].apply(encode_Thal)
data = pd.concat([data, pd.get_dummies(data['Thal'], prefix = 'Thal')], axis = 1)
data.drop('Thal', axis = 1, inplace = True)

data.head()

In [ ]:
Pain_dict = {ChestPain:i for i,ChestPain in enumerate(data['ChestPain'].unique())}

def encode_ChestPain(ChestPain):
  return Pain_dict[ChestPain]

data['ChestPain'] = data['ChestPain'].apply(encode_ChestPain)
data = pd.concat([data, pd.get_dummies(data['ChestPain'], prefix = 'ChestPain')], axis = 1)

data.drop('ChestPain', axis = 1, inplace = True)

data.head()

In [ ]:
y = data['AHD']
data.drop('AHD', axis = 1, inplace = True)
data.shape

data = data.to_numpy().astype('float32')
y = y.to_numpy().astype('float32')

## Model Construction

In [ ]:
input_layer = layers.Input(shape = (None, 19))
x = layers.Dense(100, activation = 'relu')(input_layer)
x = layers.Dropout(.2)
x = layers.Dense(100, activation = 'relu')(input_layer)
x = layers.Dense(100, activation = 'relu')(input_layer)
x = layers.Dense(1, activation='sigmoid')(x)
model = Model(input_layer, x)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Training

In [ ]:
hist = model.fit(data, y, epochs=200, batch_size=32, validation_split=0.2)